In [12]:
# import secrets and tokens from config.py
from config import client_id, client_secret, refresh_token

import requests
import urllib3

# import packages for data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': "refresh_token",
    'f': 'json'
}

In [13]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

Requesting Token...

Access Token = 428643c2a40d2d699ad8fc46662bfb50d645fd61



In [14]:
activities = pd.json_normalize(my_dataset)

In [15]:
activities.columns 

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'has_heartrate', 'average_heartrate', 'max_heartrate',
       'heartrate_opt_out', 'display_hide_heartrate_option', 'elev_high',
       'elev_low', 'upload_id', 'upload_id_str', 'external_id',
       'from_accepted_tag', 'pr_count', 'total_photo_count', 'has_kudoed',
       'suffer_score', 'athlete.id', 'athlete.resource_state', 'map.id',
       'map.summary_polyline', 'map.resource_state', 'average_cadence'],
      dtype='object')

In [16]:
# Break date into start time and date
activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])
activities['start_time'] = activities['start_date_local'].dt.time
#activities['start_date_local'] = activities['start_date_local'].dt.date
activities.head(5)

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,total_photo_count,has_kudoed,suffer_score,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_cadence,start_time
0,2,New Year's Run with 9run6,4064.1,1328,1328,6.6,Run,Run,0.0,8321656559,...,0,False,43.0,43537797,1,a8321656559,u_hjFdiqdVb@D|@PzBl@|@\l@Hj@Fr@LpBh@ZFtAd@f@Ln...,2,NaN,08:10:42
1,2,22 in the rain for 2022,4266.1,1324,1324,3.0,Run,Run,0.0,8318206059,...,0,False,58.0,43537797,1,a8318206059,megjFpvqdVTh@Lf@Fr@Cb@R^Jh@F`@Bh@DTNRF@TPd@Lf@...,2,NaN,12:08:51
2,2,Treadmill work,1609.3,600,600,0.0,Run,Run,0.0,8314641334,...,0,False,NaN,43537797,1,a8314641334,,2,NaN,18:30:31
3,2,Stationary Bike,6437.4,769,769,0.0,Ride,Ride,10.0,8314632114,...,0,False,NaN,43537797,1,a8314632114,,2,NaN,18:06:58
4,2,Treadmill work,1609.3,720,720,0.0,Run,Run,0.0,8314643357,...,0,False,NaN,43537797,1,a8314643357,,2,NaN,18:11:12


In [17]:
activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])

In [18]:
activities['start_date_local']

0    2023-01-01 08:10:42+00:00
1    2022-12-31 12:08:51+00:00
2    2022-12-30 18:30:31+00:00
3    2022-12-30 18:06:58+00:00
4    2022-12-28 18:11:12+00:00
                ...           
95   2020-01-18 15:06:00+00:00
96   2020-01-04 18:40:49+00:00
97   2019-12-13 12:00:33+00:00
98   2019-11-03 15:14:00+00:00
99   2019-09-10 15:39:04+00:00
Name: start_date_local, Length: 100, dtype: datetime64[ns, UTC]

In [19]:
activities_time_filtered = activities.copy()

In [20]:
# Filter down to activities within the last 30 days
activities_time_filtered = activities_time_filtered.query("start_date_local > '2022-12-15 08:10:42+00:00'")

In [21]:
len(activities_time_filtered)

9

In [22]:
# import modules
import os
import time
import matplotlib.pyplot as plt
import folium
import polyline
import base64
from tqdm import tqdm

# add decoded summary polylines
activities_time_filtered['map.polyline'] = activities_time_filtered['map.summary_polyline'].apply(polyline.decode)
print("turkey")
#####################
# define function to get elevation data using the open-elevation API
def get_elevation(latitude, longitude):
    base_url = 'https://api.open-elevation.com/api/v1/lookup'
    payload = {'locations': f'{latitude},{longitude}'}
    r = requests.get(base_url, params=payload).json()['results'][0]
    return r['elevation']
# get elevation data
elevation_data = list()
for idx in tqdm(activities_time_filtered.index):
    activity = activities_time_filtered.loc[idx, :]
    elevation = [get_elevation(coord[0], coord[1]) for coord in activity['map.polyline']]
    elevation_data.append(elevation)

# add decoded summary polylines
activities['map.polyline'] = activities['map.summary_polyline'].apply(polyline.decode)
print("turkey")
#####################
# define function to get elevation data using the open-elevation API
def get_elevation(latitude, longitude):
    base_url = 'https://api.open-elevation.com/api/v1/lookup'
    payload = {'locations': f'{latitude},{longitude}'}
    r = requests.get(base_url, params=payload).json()['results'][0]
    return r['elevation']
# get elevation data
elevation_data = list()
for idx in tqdm(activities.index):
    activity = activities.loc[idx, :]
    elevation = [get_elevation(coord[0], coord[1]) for coord in activity['map.polyline']]
    elevation_data.append(elevation)

In [23]:
# add elevation data to dataframe
# activities['map.elevation'] = elevation_data

In [24]:
# Make a copy of activities DataFrame for testing.
activities_copy = activities.copy()

In [25]:
# filter down to Ride, Run, and Swim activities
activities_copy = activities_copy.query("type == 'Ride' | type == 'Run' | type == 'Swim'")
print(len(activities_copy))

76


In [26]:
# convert data types
activities_copy.loc[:, 'start_date'] = pd.to_datetime(activities_copy['start_date']).dt.tz_localize(None)
activities_copy.loc[:, 'start_date_local'] = pd.to_datetime(activities_copy['start_date_local']).dt.tz_localize(None)
# convert values
activities_copy.loc[:, 'distance'] /= 1609.344 # convert from meters to miles
activities_copy.loc[:, 'average_speed'] *= 2.23693629 # convert from meters/second to miles/hour
activities_copy.loc[:, 'max_speed'] *= 2.23693629 # convert from meters/second to miles/hour
# set index
activities_copy.set_index('start_date_local', inplace=True)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_2404\363641948.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  activities_copy.loc[:, 'start_date'] = pd.to_datetime(activities_copy['start_date']).dt.tz_localize(None)


In [27]:
miles_per_activity = pd.DataFrame(activities_copy.groupby(['type'])['distance'].sum().reset_index())
miles_per_activity

,type,distance
0,Ride,70.553157
1,Run,135.460101
2,Swim,0.994194


In [28]:
type(miles_per_activity)

pandas.core.frame.DataFrame

In [36]:
# Set monthly goals in miles per activity
practice_sessions = 8

# 20 mile ride per session
ride_goal = 20 * practice_sessions
# 4 mile run per session
run_goal = 4 * practice_sessions
# half-mile swim per session
swim_goal = 0.5 * practice_sessions
print(ride_goal, run_goal, swim_goal)
exercise_type_list = ['Ride','Run','Swim']
goal_list = [ride_goal, run_goal, swim_goal]


160 32 4.0


In [37]:
dictionary = dict(zip(exercise_type_list, goal_list))
print(dictionary)

{'Ride': 160, 'Run': 32, 'Swim': 4.0}


In [40]:
#https://stackoverflow.com/questions/17839973/constructing-pandas-dataframe-from-values-in-variables-gives-valueerror-if-usi
#keys = ['a', 'b', 'c']
#values = [1, 2, 3]
#dictionary = dict(zip(keys, values))

# Make a df of the goal mileage
#pd.Series(data).to_frame()
goal_df = pd.Series(dictionary).to_frame()

In [41]:
goal_df

,0
Ride,160.0
Run,32.0
Swim,4.0


In [30]:
dif_ride_miles = ride_goal - miles_per_activity['distance'][0]
dif_ride_miles

89.44684293724649

In [31]:
print("You need to ride " + str(dif_ride_miles) + " to meet your goal this month!")

You need to ride 89.44684293724649 to meet your goal this month!


In [32]:
# filter activities based on start time

In [33]:
# Build out simple graph of miles run versus miles goal
import plotly.express as px


In [34]:
fig = px.bar(miles_per_activity, x = 'type', y = 'distance')
fig.show()